In [1]:
from ontolearn.knowledge_base import KnowledgeBase
from owlapy.render import DLSyntaxObjectRenderer

In [2]:
import json

In [20]:
def statistics(kb: str):
    renderer = DLSyntaxObjectRenderer()
    KB = KnowledgeBase(path=f'datasets/{kb}/{kb}.owl')
    atomic_concepts = [renderer.render(c) for c in KB.ontology().classes_in_signature()]
    role_names = [rel.get_iri().get_remainder() for rel in KB.ontology().object_properties_in_signature()]
    vocab = atomic_concepts + role_names + ['⊔', '⊓', '∃', '∀', '¬', '⊤', '⊥', '.', ' ', '(', ')']
    vocab_card = len(vocab)
    with open(f'datasets/{kb}/Train_data/Data.json') as lp:
        data = json.load(lp)
    with open(f'datasets/{kb}/Test_data/Data.json') as lp:
        data_test = json.load(lp)
    train_card = len(data)
    lp_card = len(data_test)
    print(f"|Vocab|: {vocab_card}, |Train|: {train_card}, |LP|: {lp_card}")

In [22]:
statistics('carcinogenesis')

|Vocab|: 157, |Train|: 9662, |LP|: 98


In [23]:
statistics('mutagenesis')

|Vocab|: 102, |Train|: 2138, |LP|: 22


In [24]:
statistics('semantic_bible')

|Vocab|: 88, |Train|: 2479, |LP|: 26


In [25]:
statistics('vicodi')

|Vocab|: 215, |Train|: 15533, |LP|: 157


In [10]:
from owlapy.parser import DLSyntaxParser
from utils.evaluator import Evaluator

In [18]:
def recalculate_metrics(kb):
    KB = KnowledgeBase(path=f'datasets/{kb}/{kb}.owl')
    namespace = KB.ontology()._onto.base_iri
    if kb == 'family-benchmark':
        namespace = 'http://www.benchmark.org/family#'
    if kb == 'vicodi':
        namespace = 'http://vicodi.org/ontology#'
    print()
    evaluator = Evaluator(KB)
    dl_parser = DLSyntaxParser(namespace = namespace)
    All_individuals = set(KB.individuals())
    for model in ['eltl', 'celoe']:
        print(f'Model: {model.upper()}\n')
        with open(f'datasets/{kb}/Results/concept_learning_results_{model}.json') as file:
            results = json.load(file)
        new_results = {'F-measure': [], 'Accuracy': [], 'Runtime': [], 'Prediction': [], 'Length': [], 'Learned Concept': []}
        for key in results:
            if key not in ['F-measure', 'Accuracy']:
                new_results[key] = results[key]
        prediction = results['Prediction']
        for pred, target in zip(prediction, results['Learned Concept']):
            target_expression = dl_parser.parse_expression(target)
            positive_examples = {ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(target_expression)}
            negative_examples = All_individuals-positive_examples
            Pred = dl_parser.parse_expression(pred)
            acc, f1 = evaluator.evaluate(Pred, positive_examples, negative_examples)
            new_results['F-measure'].append(f1)
            new_results['Accuracy'].append(acc)
            print(f'F1: {f1}, Acc: {acc}')